Given a Run and a set of events of this Run, this NB searches and saves them. Only S2 sipmt data and pmt total energy data is saved. Inputs needed are:

- Run number.
- Events number to select.
- The number of the last partition in the Run that you want to search.
- Directory where pmaps files are saved.

In [1]:
import os

import warnings

import invisible_cities.io.dst_io as ht

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#INPUTS:

run       = 6206
npart     = 40000  #number of the last partition, or higher
directory = "/home/gonzalo/Documentos/NEXT/DATOS/{}/hdst".format(run)

n_ev  = 200 #number of events in each file

In [3]:
#EVENTS TO SELECT (choose one)

#cs
cs_evt = np.load("cs_evts_{}.npz".format(run))['A_evtnum']

#descape
descape_evt = np.load("descape_evts_{}.npz".format(run))['A_evtnum']

#th
th_evt = np.load("photopeakTh_evts_{}.npz".format(run))['A_evtnum']

evts = th_evt
evts.sort()     #events sorting (this is not compulsory)

In [4]:
#evts.sort()
#evts_part  = [evts[n_ev*i:(i+1)*n_ev] for i in range(0, int(len(evts)/n_ev))] + [evts[n_ev * int(len(evts)/n_ev):]]
#partitions = ["{:04}".format(i) for i in range(0, npart+1)]

In [4]:
#SEARCHER AND DATAFRAME CREATOR

#COMMENT: do not mistake the number of partition of pmap, with the number of chunk of the selected data

part = 0
n_chunk = int(len(evts)/n_ev) + 1 #total number of chunk

for j in range(0, n_chunk): #j is the number of chunk
    file = 'Thphotopeak_hdst_{}_{}.h5'.format(run, j) #filename of the saved data
    i = 0
    partitions = ["{:04}".format(i) for i in range(int(part), npart+1)]
    for part in partitions:
        
        try: 
            #partition data as dataframe            
            filename = '{}/hdst_{}_{}_v0.9.7-19-g574c657_20180710_krth1300.h5'.format(directory, part, run)
        
            with warnings.catch_warnings():
                warnings.simplefilter("error")
                dst = ht.load_dst(filename, 'RECO', 'Events')
                
            #event search
            ar1 = np.unique(dst['event'])
            coinc = np.intersect1d(ar1, evts, True) #coincident events
        
            for event in coinc:
                dst_ev   = dst[dst.event == event]
                
                dst_ev   .to_hdf(file, key = 'hdst'   , append = True)
                
                #this is useful if some of the coinc should be saved in a diferent chunk
                evts = np.delete(evts, np.argwhere(evts == event))
                
                i+=1
                print('Events saved in chunk {:02} of {:02}: {:03} %'.format(j, n_chunk-1
                                                                              , int(i*100/n_ev)), end = '\r')
                if i >= n_ev: break
                    
            if i>= n_ev: break
                
        except UserWarning: continue 
    if len(evts)==0: break

In [47]:
#READ (for comprobation purposes)
file = 'Csphotopeak_hdst_{}_{}.h5'.format(run, 19)
hdf = pd.HDFStore(file)

In [48]:
df = hdf['hdst']

In [49]:
len(np.intersect1d(np.unique(df['event']), cs_evt[20*n_ev:21*n_ev]))

1

In [50]:
np.unique(df['event'])

array([1243309, 1244302, 1244443, 1245244, 1245326, 1245939, 1246785,
       1247483, 1247520, 1247535, 1247623, 1247849, 1248473, 1248649,
       1248783, 1248855, 1248922, 1249327, 1249366, 1249718, 1250069,
       1250090, 1250370, 1250674, 1250898, 1252059, 1252832, 1253011,
       1253206, 1253208, 1253278, 1253416, 1253538, 1253635, 1254155,
       1254298, 1254562, 1254813, 1256008, 1256043, 1256384, 1257021,
       1257024, 1257337, 1257427, 1257440, 1257690, 1258833, 1258858,
       1259859, 1261092, 1261182, 1261226, 1261341, 1262185, 1263448,
       1263505, 1264055, 1264582, 1265456, 1265541, 1265563, 1265920,
       1266404, 1267148, 1267297, 1267607, 1267847, 1268014, 1268327,
       1268669, 1268834, 1268970, 1270177, 1270205, 1271108, 1271113,
       1271678, 1271696, 1271731, 1271745, 1271809, 1271845, 1272031,
       1274130, 1275039, 1275461, 1275782, 1275885, 1276313, 1276777,
       1276917, 1276923, 1276972, 1277321, 1277428, 1278230, 1278302,
       1278306, 1278

In [46]:
cs_evt[20*n_ev:21*n_ev]

array([1306660, 1306926, 1307507, 1307897, 1308007, 1308088, 1308339,
       1308677, 1308820, 1308927, 1309077, 1309213, 1309488, 1310049,
       1310770, 1311451, 1311607, 1311707, 1311774, 1311792, 1312068,
       1312634, 1312694, 1313536, 1313554, 1314154, 1314409, 1314727,
       1314962, 1315164, 1315444, 1315549, 1315582, 1315662, 1316084,
       1316157, 1316210, 1316220, 1316308, 1316614, 1316622, 1316677,
       1316726, 1317398, 1318497, 1318798, 1318985, 1319138, 1319379,
       1319444, 1319577, 1320060, 1320092, 1320168, 1320416, 1320598,
       1320846, 1320941, 1321365, 1321400, 1321658, 1322141, 1322168,
       1322531, 1322660, 1322787, 1322852, 1323016, 1323032, 1323170,
       1323281, 1324232, 1324236, 1324333, 1324365, 1324419, 1325337,
       1325579, 1325644, 1325990, 1326047, 1327161, 1327325, 1327626,
       1327740, 1328969, 1329057, 1329066, 1329853, 1329988, 1330216,
       1330220, 1330496, 1330516, 1330621, 1330976, 1331042, 1331118,
       1331139, 1331